In [8]:
#Should be finished running at 11:20 am
%pip install tensorflow
%pip install --upgrade Pillow
%pip install -U scikit-image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
import os
import glob
import numpy as np 
import random
import cv2
import tensorflow as tf
from PIL import Image
from skimage import io
import sagemaker
import boto3
import pandas as pd
import numpy
from torch.utils.data import Dataset, DataLoader

In [10]:
class Net(nn.Module):


  def __init__(self):

    super(Net, self).__init__()

    input_width = 32
    input_height = 32


    self.net = nn.Sequential()


    #Activation map of size: 3 * input_width * input_height
    self.net.add_module('cv1', nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding =  1, dilation = 1))

    #Activation map of size: 64 * input_widht * input_height
    self.net.add_module('r11', nn.ReLU())

    #Activation map of size: 64 * input_width * input_height
    self.net.add_module('cv2', nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size 128* input_width * input_height
    self.net.add_module('rl2', nn.ReLU())

    #Activation map of size: 128 * input_width/2 * input_height/2
    self.net.add_module('mp1', nn.MaxPool2d(kernel_size = 2, stride = None, padding = 0, dilation = 1))

    #Activation map of size: 128 * input_width/2 * input_height/2
    self.net.add_module('cv3', nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size: 256 * input_width/2 * input_height/2
    self.net.add_module('rl3', nn.ReLU())

    #Activation map of size 256 * input_width/2 * input_height /2
    self.net.add_module('cv4', nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size 512 * input_width/2 * input_height/2
    self.net.add_module('rl4', nn.ReLU())

    #Activation map of size 512 * input_width/2 * input_height/2
    self.net.add_module('mp2', nn.MaxPool2d(kernel_size = 2, stride = None, padding = 0, dilation = 1))

    #Activation map of size 512 * input_width/4 * input_height/4
    #self.net.add_module('cv5', nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #Activation map of size 512 * input_width/4 * input_height/4
    #self.net.add_module('rl5', nn.ReLU())

    #activation map of size 512 * input_width/4 * input_height/4
    #self.net.add_module('cv6', nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, dilation = 1))

    #activation map of size 512 * input_width/4 * input_height/4
    #self.net.add_module('rl6', nn.ReLU())



    #Multilayer perceptron
    self.net.add_module('dp1', nn.Dropout2d(p = 0.25))
    self.net.add_module('fl1', nn.Flatten())
    self.net.add_module('fc1', nn.Linear(in_features = 771840, out_features = 128))
    self.net.add_module('rl7', nn.ReLU())
    self.net.add_module('fc2', nn.Linear(in_features = 128, out_features = 4))
    self.net.add_module('sm1', nn.LogSoftmax(dim = 1))

  def forward(self, x):
    return self.net(x.float())

model = Net()

optimizer = optim.Adam(model.parameters(), lr=0.1)

# Let's define a Loss function

lossfun = nn.NLLLoss()  # Use nn.CrossEntropyLoss with softmax

In [11]:

%matplotlib inline

from torchvision import datasets, transforms
import os

imgs = datasets.ImageFolder('Data/Padded', transform = transforms.ToTensor())

f = open("Data/Target/y_target.txt")

PATH = "Data"

image_path = os.path.join(PATH, 'Resized', '*')
image_paths = sorted(glob.glob(image_path)) 
len(image_paths)


964

In [12]:
for i in range(5):
    img = io.imread(image_paths[i])
    print(img.shape)
    

(180, 270, 3)
(180, 270, 3)
(180, 270, 3)
(180, 270, 3)
(180, 270, 3)


In [13]:
#Attempting to make a custom dataset, so that we can use the below training algorthm. 

class TrashDataset():
    def __init__(self, target_vector_file, train, transform = None):
        dataset = {}
        target_file = open("Data/Target/y_target_with_name.txt", "r")
        img_name = target_file.readline()
        i = 0
        classifications = []
        img_names = []
        while(img_name != ""):
            img_names.append(str(img_name).strip())
            classifications.append(str(target_file.readline()).strip())
            img_name = target_file.readline()
        target_file.close()
        
        dataset = []
        
        for i in range(len(img_names)):
            dataset.append( (img_names[i], classifications[i]))
        
        self.dataframe = pd.DataFrame(dataset)
        self.dataframe = self.dataframe.sample(frac = 1, random_state = 42)
        
        if train:
            self.dataframe = self.dataframe.head(771)
        
        else:
            self.dataframe = self.dataframe.tail(193)
            
        print(self.dataframe)
        self.transform = transform
        
        
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__ (self, index):
        if torch.is_tensor(index):
            idx = idx.tolist(index)
        img_name = self.dataframe.iloc[index, 0]
        img = io.imread("Data/Resized/" + img_name[:-5] + ".jpg")
        classification = self.dataframe.iloc[index, 1]
        sample = {"image": img, "classification": classification}
        sample = self.transform(sample)
        return sample
        

        
class ToTensor(object):
    #Class to turn nparrays to tensor
    def __call__(self, sample):
        image, classification= sample["image"], sample["classification"]
        image = image.transpose((2, 0, 1))
        return {"image": torch.from_numpy(image), "classification": int(classification)}

In [14]:
train_dataset = TrashDataset("Data/Target/y_target_with_name.txt", train = True, transform = ToTensor())
test_dataset = TrashDataset("Data/Target/y_target_with_name.txt", train = False, transform = ToTensor())


                                                     0  1
760  fe15319d-7d89-4df6-82e5-52b4f6c001af_fa3cba4e-...  2
884  ff4e61c6-96aa-4b02-aa12-1b7b5881bfb0_a821fad2-...  2
938  ffc6ade2-5bc5-45ce-90aa-d78b591e1b9d_2d5a0bdd-...  2
901  ff6d9c14-1b58-46eb-a133-1c36b9b7f675_c9be3801-...  1
622  5e9875ff-4b9a-4b90-b190-05bf6c2aa0ac_b23a4bc2-...  2
..                                                 ... ..
619  5e932284-4d4e-4fca-aef0-ad901d0792ee_448f8405-...  2
823  feac2bb4-ee0a-463c-86c6-9eb76a015124_365d330f-...  1
645  5ed1a95e-da7a-46d9-afb3-6920244b439c_80c9f4e3-...  1
866  ff1fd6bd-69e7-4ebf-a938-9b6aff1c4628_b083732a-...  1
556  5ded1e7d-b110-4969-898c-5437201eba56_599786ed-...  1

[771 rows x 2 columns]
                                                     0  1
922  ff931781-06e1-41d7-8420-81191ae1b597_528807fd-...  2
577  5e10cede-14d2-45a4-914f-8a13c146425e_9444fe23-...  1
85   1c29a97b-794d-4402-b358-572dd8dab846_08205073-...  1
242  5aa36e28-c878-4ae5-ac9a-598319f4751d_15e245

In [15]:
#Training the model, (here we go!)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 4)

def train(model, train_loader, epochs):
    model.train()

    # Define train epochs

    for epoch in range(epochs):
        

        # iterate through train dataset
        epoch_loss = []
        epoch_accu = []

        for batch_idx, sample in enumerate(train_loader):
            
            #data, target = data.to(device), target.to(device)
            
            data = sample["image"]
            target = sample["classification"]

            #print(data.shape)

            # get output
            output = model(data)

            # compute loss function
            loss = lossfun(output, target)

            # backward pass
            optimizer.zero_grad()
            loss.backward()

            # run optimizer 
            optimizer.step()

            # bookkeeping
            accuracy = (output.argmax(-1) == target).float().mean()
            epoch_loss.append(loss.item())
            epoch_accu.append(accuracy.item())

            if batch_idx % 12 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item(), accuracy.item()))

        print('Train Epoch: {}\tAverage Loss: {:.6f}\tAverage Accuracy: {:.2f}'.format(
            epoch+1, sum(epoch_loss)/len(epoch_loss), sum(epoch_accu)/len(epoch_accu)))
            
    # save network
    
    return epoch_loss, epoch_accu

epoch_loss, epoch_accu = train(model, train_loader, epochs=50)


[2021-06-17 01:37:59.799 pytorch-1-6-gpu-py3-ml-m5-12xlarge-7d7e51df9510cb050180ece741a1:35 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-06-17 01:37:59.825 pytorch-1-6-gpu-py3-ml-m5-12xlarge-7d7e51df9510cb050180ece741a1:35 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Train Epoch: 1 [0/771 (0%)]	Loss: 1.816106	Accuracy: 0.12
Train Epoch: 1 [384/771 (48%)]	Loss: 18.908144	Accuracy: 0.28
Train Epoch: 1 [72/771 (96%)]	Loss: 1.181961	Accuracy: 0.33
Train Epoch: 1	Average Loss: 35118782943.134315	Average Accuracy: 0.32
Train Epoch: 2 [0/771 (0%)]	Loss: 1.223186	Accuracy: 0.28
Train Epoch: 2 [384/771 (48%)]	Loss: 1.335196	Accuracy: 0.22
Train Epoch: 2 [72/771 (96%)]	Loss: 1.055981	Accuracy: 1.00
Train Epoch: 2	Average Loss: 1.277965	Average Accuracy: 0.37
Train Epoch: 3 [0/771 (0%)]	Loss: 1.368985	Accuracy: 0.28
Train Epoch: 3 [384/771 (48%)]	Loss: 1.305671	Accuracy: 0.41
Train Epoch: 3 [72/771 (96%

In [16]:
def test(test_loader):
    
    total_loss = []
    total_accu = []
    
    print("starting to test...")
    
    for batch_idx, sample in enumerate(test_loader):
        
            print("Testing batch index: ", batch_idx)
            

            #data, target = data.to(device), target.to(device)
            
            data = sample["image"]
            target = sample["classification"]

            #print(data.shape)

            # get output
            output = model(data)

            # compute loss function
            loss = lossfun(output, target)

            # bookkeeping
            accuracy = (output.argmax(-1) == target).float().mean()
            total_loss.append(loss.item())
            total_accu.append(accuracy.item())
            

    print('Average Loss: {:.6f}\tAverage Accuracy: {:.2f}'.format(
            sum(total_loss)/len(total_loss), sum(total_accu)/len(total_accu)))
            
    
    return total_loss, total_accu

test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

test(test_loader)

starting to test...
Testing batch index:  0
Testing batch index:  1
Testing batch index:  2
Testing batch index:  3
Testing batch index:  4
Testing batch index:  5
Testing batch index:  6
Average Loss: 1.264133	Average Accuracy: 0.30


([1.2168331146240234,
  1.2642390727996826,
  1.2565412521362305,
  1.2239166498184204,
  1.306265115737915,
  1.4071767330169678,
  1.1739568710327148],
 [0.28125, 0.34375, 0.4375, 0.34375, 0.375, 0.34375, 0.0])

In [213]:
#Train attempt 2

#Set up permissions and enviroment variables. 

import sagemaker
from sagemaker import get_execution_role
role = get_execution_role
print(role)

sess = sagemaker.Session()
bucket = "e-team6"
prefix = "ic-fulltraining"

<function get_execution_role at 0x7fabf2415bf8>


In [12]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, "image-classification", repo_version = "latest")



The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


In [13]:
#Data Preperation

import boto3 

#Function to transfer data to s3 for training. 

def upload_to_s3(channel, file):
    s3 = boto3.resource("s3")
    data = open(file, "rb")
    key = channel + "/" + file
    s3.Bucket(bucket).put_object(Key = key, Body = data)

In [14]:
#Split data into to channels -- train and test

s3train = "s3://{}/{}/train".format(bucket, prefix)
s3validation = "s3://{}/{}/validation".format(bucket, prefix)

In [15]:
#Upload images into validation and training channels

In [16]:
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)

ic = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count = 1, 
    train_instance_type = "m1.p2.xlarge",
    train_volume_size = 50, 
    train_max_run = 360000,
    input_model = "file",
    output_path = s3_output_location, 
    sagemaker_session = sess

    )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
#Set up hyperparmeters in the algorithm.

ic.set_hyperparameters(
    num_layers = 18, 
    image_shape = "3, 5312, 4032",
    num_classes = 4, 
    num_training_examples = 400,
    mini_batch_size = 32,
    epochs = 10,
    learning_rate = 0.1,
    top_k = 2#I have no idea what this parameter does
    )

In [18]:
train_data = sagemaker.inputs.TrainingInput(
    s3train, 
    distribution = "FullyReplicated",
    content_type = "image/jpeg",
    s3_data_type = "S3Prefix",
    )

validation_data = sagemaker.inputs.TrainingInput(
    s3validation, 
    distribution = "FullyRepliatied",
    content_type = "image/jpeg", 
    s3_data_type = "S3Prefix"
    )

data_channels = {"train": train_data, "validation": validation_data}

In [19]:
import os
import glob

PATH = "train"
image_path = os.path.join(PATH, 'images', '*')
image_paths = sorted(glob.glob(image_path)) 

for path in image_paths:
    upload_to_s3("train", path)
    
PATH = "valid"    
image_path = os.path.join(PATH, "images", "*")
imagePaths = sorted(glob.glob(image_path))

for path in image_paths:
    upload_to_s3("validation", path)
    

In [20]:
ic.fit(inputs=data_channels, logs=True)

TypeError: argument of type 'function' is not iterable